In [ ]:
%load_ext watermark


In [ ]:
import itertools as it
import os

from downstream import dstream
import matplotlib as mpl
import more_itertools as mit
import numpy as np
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp

import pylib  # noqa: F401


In [ ]:
%watermark -diwmuv -iv


In [ ]:
tp.save[".pgf"] = False
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2024-10-20-qos-dstream-vs-naive-tilted"
)
teeplot_subdir


In [ ]:
num_items = 10_000


## Define


In [ ]:
def calc_qos_from_segment_lengths(segment_lengths: list[int]) -> float:
    segment_total = sum(segment_lengths)
    return max(
        (segment_length - 1) / ((segment_total - cumulative) or 1)
        for cumulative, segment_length in zip(
            it.accumulate([0, *segment_lengths]),
            [*segment_lengths, 1],
        )
    )


In [ ]:
def calc_max_gaps_naive_doubling_tilted(
    buffer_size: int, num_ingests: int
) -> list[int]:
    segment_lengths = []
    max_gaps = [0]
    for i in range(num_ingests):

        if (len(segment_lengths) == buffer_size):
            segment_lengths = [
                a + b
                for a, b in mit.batched(segment_lengths, 2)
            ]
        segment_lengths.append(1)

        assert sum(segment_lengths) == i + 1
        max_gaps.append(calc_qos_from_segment_lengths(segment_lengths))

    return max_gaps[:-1]


In [ ]:
def calc_max_gaps_dstream(buffer_size: int, num_items: int) -> list[int]:
    return [
        calc_qos_from_segment_lengths(
            [b - a for a, b in mit.pairwise(
            sorted(
                dstream.tilted_algo.lookup_ingest_times_eager(
                    buffer_size, i + 1
                ),
            ))],
        )
        if i >= buffer_size
        else 0
        for i in range(num_items)
    ]


In [ ]:
def calc_max_gaps_zhao_tilted(buffer_size: int, num_items: int) -> list[int]:
    segment_lengths = []
    max_gaps = [0]
    for k in range(num_items):

        segment_lengths.append(1)
        for i, j in it.pairwise(reversed(range(len(segment_lengths) - 1))):
            if segment_lengths[i] == segment_lengths[j]:
                segment_lengths[i] += segment_lengths[j]
                del segment_lengths[j]
                break

        assert sum(segment_lengths) == k + 1
        max_gaps.append(calc_qos_from_segment_lengths(segment_lengths))

    return max_gaps[:-1]


In [ ]:
def calc_max_gaps_zhao_tilted_full(
    buffer_size: int, num_items: int
) -> list[int]:
    bucket_sizes = [buffer_size, 0]
    buffer = [*range(buffer_size)]
    max_gaps = [0]


    S = buffer_size
    w = bucket_sizes
    for k in range(num_items):
        if k < S:
            assert buffer[k] == k
        else:
            bucket_sizes[0] += 1
            i = S
            j = 0
            while w[j] <= w[j + 1]:
                i -= w[j]
                j += 1
                if j == len(w) - 1:
                    w.append(0)

            assert 0 <= i <= S
            w[j] -= 2
            w[j + 1] += 1
            for n in range(i - w[j], S - 1):
                assert 0 < n < S - 1
                buffer[n] = buffer[n + 1]
            buffer[S - 1] = k

        assert buffer == sorted(buffer)
        assert sum(bucket_sizes) == buffer_size

        segment_lengths = [
            b - a for a, b in mit.pairwise(sorted(buffer))
        ]
        assert (
            sum(segment_lengths)
            == buffer_size + max(k - buffer_size + 1, 0) - 1
        )
        max_gaps.append(
            calc_qos_from_segment_lengths(segment_lengths),
        )

    return max_gaps[:-1]


## Example Plot


In [ ]:
def make_df(buffer_size: int) -> pl.DataFrame:
    return pl.concat(
        [
            pl.DataFrame(
                {
                    "Algorithm": "doubling tilted",
                    "Gap Size Cost": calc_max_gaps_naive_doubling_tilted(
                        buffer_size, num_items
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
            pl.DataFrame(
                {
                    "Algorithm": "dstream tilted",
                    "Gap Size Cost": calc_max_gaps_dstream(
                        buffer_size, num_items
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
            pl.DataFrame(
                {
                    "Algorithm": "zhao tilted",
                    "Gap Size Cost": calc_max_gaps_zhao_tilted(
                        buffer_size, num_items
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
            pl.DataFrame(
                {
                    "Algorithm": "zhao tilted full",
                    "Gap Size Cost": calc_max_gaps_zhao_tilted_full(
                        buffer_size, num_items
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
        ]
    )


In [ ]:
for buffer_size in [64, 256, 1024, 4096]:
    df = make_df(buffer_size)
    for rc in [{}, {"font.family": "serif"}]:
        with mpl.rc_context(rc=rc):
            with tp.teed(
                sns.relplot,
                df,
                x="Num Items Ingested",
                y="Gap Size Cost",
                hue="Algorithm",
                hue_order=[
                    "dstream tilted",
                    "doubling tilted",
                    "zhao tilted",
                    "zhao tilted full",
                ],
                style="Algorithm",
                aspect=2.8,
                kind="line",
                height=1.8,
                palette="Set2",
                teeplot_outattrs={
                    "buffer_size": buffer_size,
                    **rc,
                },
                teeplot_subdir=teeplot_subdir,
            ) as g:
                sns.move_legend(
                    g,
                    "lower center",
                    bbox_to_anchor=(0.4, 1),
                    ncol=3,
                    title=None,
                    frameon=False,
                )

            dfp = df.pivot(
                index="Num Items Ingested",
                on="Algorithm",
                values="Gap Size Cost",
            ).to_pandas()
            dfp["ratio"] = (
                dfp["zhao tilted full"] / dfp["dstream tilted"]
            )
            inf, nan = np.inf, np.nan
            print(
                f"{dfp['ratio'].min()=:.2f}",
                f"{dfp['ratio'].replace(inf, nan).dropna().mean()=:.2f}",
                sep="\n",
            )
            with tp.teed(
                sns.relplot,
                dfp,
                x="Num Items Ingested",
                y="ratio",
                aspect=2.8,
                kind="line",
                height=1.8,
                palette="Set2",
                teeplot_outattrs={
                    "buffer_size": buffer_size,
                    **rc,
                },
                teeplot_subdir=teeplot_subdir,
            ) as g:
                g.set(ylim=(0, 1))
                g.set(ylabel="Gap Size Cost\nZhao Full:DStream")
